In [1]:
import os
import numpy as np
import utils

CMD=["perl", "../english_testbed/data/2015/evaluate/correlation-noconfidence.pl"]
CMD2016=["perl","../english_testbed/data/2016/test/correlation-noconfidence.pl"]
class Opts:
    verbose=False
    filter_test=".*"
    
opts=Opts()

In [2]:
TRAIN_2015_DATADIR="english_testbed/analysis/2015/training/results/"
TEST_2015_DATADIR="english_testbed/analysis/2015/testing/results"
TEST_2016_DATADIR="english_testbed/analysis/2016/testing/results"
TEST_2015_DIR="english_testbed/data/2015/test/"
TEST_2016_DIR="english_testbed/data/2016/test/"
TRAIN_2015_NAMEFILES=[
  "STS.2012.test.input.MSRpar.txt",
  "STS.2012.test.input.MSRvid.txt",
  "STS.2012.test.input.SMTeuroparl.txt",
  "STS.2012.test.input.surprise.OnWN.txt",
  "STS.2012.test.input.surprise.SMTnews.txt",
  "STS.2012.train.input.MSRpar.txt",
  "STS.2012.train.input.MSRvid.txt",
  "STS.2012.train.input.SMTeuroparl.txt",
  "STS.2013.test.input.FNWN.txt",
  "STS.2013.test.input.headlines.txt",
  "STS.2013.test.input.OnWN.txt",
  "STS.2013.test.input.SMT.txt",
  "STS.2014.test.input.deft-forum.txt",
  "STS.2014.test.input.deft-news.txt",
  "STS.2014.test.input.headlines.txt",
  "STS.2014.test.input.images.txt",
  "STS.2014.test.input.OnWN.txt",
  "STS.2014.test.input.tweet-news.txt"]
TEST_2015_NAMEFILES=[
  "STS.input.answers-forums.txt",
  "STS.input.answers-students.txt",
  "STS.input.belief.txt",
  "STS.input.headlines.txt",
  "STS.input.images.txt"]
TEST_2016_NAMEFILES=[
  "STS2016.input.answer-answer.txt",
  "STS2016.input.headlines.txt",
  "STS2016.input.plagiarism.txt",
  "STS2016.input.postediting.txt",
  "STS2016.input.question-question.txt"
]

RAW_2016_TRAINDIR="raw_train"

OUTPUT_DIR='output'

CROSS_DIR="crosslingual_testbed/analysis/2016/testing/results"
CROSS_2016_NAMEFILES=[
    "STS.2016.input.crosslingual-news-part1.txt",
    "STS.2016.input.crosslingual-news-part2.txt"
]



In [51]:
def read_data_file(filename,nan=False):
    GS=[]
    DATA=[]
    with open(filename) as f:
        for line in f:
            line=line.strip()
            bits=line.split()
            #print [x.split(':') for x in bits[1:]]
            line=[ float(x.split(':')[1]) for x in bits[1:]]
            line=np.nan_to_num(line)
            line[line>200]=2
            if not np.isfinite(line).all():
                print "problem"
                print line
                print np.nan_to_num(line)
                if nan:
                   continue
            GS.append(float(bits[0]))    
            DATA.append(line)
    return GS,np.array(DATA)

def read_raw_file(filename,nan=False):
    DATA=[]
    with open(filename) as f:
        for line in f:
            line=line.strip()
            bits=line.split(', ')
            line=[ float(x) for x in bits]
            line=np.nan_to_num(line)
            line[line>200]=2
            if not np.isfinite(line).all():
                print "problem"
                print line
                print np.nan_to_num(line)
                if nan:
                   continue
            DATA.append(line)
    return np.array(DATA)

def read_raw_files(filenamegs,filenamesys,nan=False):
    DATA1=[]
    DATA2=[]
    with open(filenamegs) as f, open(filenamesys) as f_:
        for line1,line2 in zip(f,f_):
            line1=line1.strip()
            if len(line1)==0:
                continue
            line=line1
            bits=line.split(', ')
            line=[ float(x) for x in bits]
            line=np.nan_to_num(line)
            line[line>200]=2
            if not np.isfinite(line).all():
                print "problem"
                print line
                print np.nan_to_num(line)
                if nan:
                   continue
            DATA1.append(line)
            line=line2
            bits=line.split(', ')
            line=[ float(x) for x in bits]
            line=np.nan_to_num(line)
            line[line>200]=2
            if not np.isfinite(line).all():
                print "problem"
                print line
                print np.nan_to_num(line)
                if nan:
                   continue
            DATA2.append(line)
    return np.array(DATA1),np.array(DATA2)


def make_data_name(dirname,filename,replace=True):
    if replace:
        filename=filename.replace(".txt",".dat")
    else:
        filename=filename+".dat"
    return os.path.join("..",dirname,filename)

def make_raw_name(dirname,filename):
    filename=filename.replace(".txt",".raw")
    return os.path.join(dirname,filename)

In [4]:
DATA_TRAIN_2015={}
DATA_TEST_2015={}
RAW_TRAIN_2015={}
RAW_TEST_2015={}

for filename in TRAIN_2015_NAMEFILES:
    filename_=make_data_name(TRAIN_2015_DATADIR,filename)
    gs,data=read_data_file(filename_,nan=True)
    DATA_TRAIN_2015[filename]=(gs,data)
    rawname=make_raw_name(RAW_2016_TRAINDIR,filename)
    cols=read_raw_file(rawname)
    RAW_TRAIN_2015[filename]=cols
    print "Shape para",filename,data.shape, cols.shape

for filename in TEST_2015_NAMEFILES:
    filename_=make_data_name(TEST_2015_DATADIR,filename)
    gs,data=read_data_file(filename_)
    DATA_TEST_2015[filename]=(gs,data)
    rawname=make_raw_name(RAW_2016_TRAINDIR,filename.replace('STS','STS.2015'))
    cols=read_raw_file(rawname)
    RAW_TEST_2015[filename]=cols
    print "Shape para",filename,data.shape, cols.shape


print "Total train data files 2015:",len(DATA_TRAIN_2015)
print "Total test  data files 2015:",len(DATA_TEST_2015)   
print "Total train raw files 2015:",len(RAW_TRAIN_2015)
print "Total test  raw files 2015:",len(RAW_TEST_2015)    

Shape para STS.2012.test.input.MSRpar.txt (750, 16) (750, 7)
Shape para STS.2012.test.input.MSRvid.txt (750, 16) (750, 7)
Shape para STS.2012.test.input.SMTeuroparl.txt (459, 16) (459, 7)
Shape para STS.2012.test.input.surprise.OnWN.txt (750, 16) (750, 7)
Shape para STS.2012.test.input.surprise.SMTnews.txt (399, 16) (399, 7)
Shape para STS.2012.train.input.MSRpar.txt (750, 16) (750, 7)
Shape para STS.2012.train.input.MSRvid.txt (750, 16) (750, 7)
Shape para STS.2012.train.input.SMTeuroparl.txt (734, 16) (734, 7)
Shape para STS.2013.test.input.FNWN.txt (189, 16) (189, 7)
Shape para STS.2013.test.input.headlines.txt (750, 16) (750, 7)
Shape para STS.2013.test.input.OnWN.txt (561, 16) (561, 7)
Shape para STS.2013.test.input.SMT.txt (750, 16) (750, 7)
Shape para STS.2014.test.input.deft-forum.txt (450, 16) (450, 7)
Shape para STS.2014.test.input.deft-news.txt (300, 16) (300, 7)
Shape para STS.2014.test.input.headlines.txt (750, 16) (750, 7)
Shape para STS.2014.test.input.images.txt (750, 1

In [5]:
X_train_2015=[]
Y_train_2015=[]
X_test_2015=[]
Y_test_2015=[]

for filename,(gs,data) in DATA_TRAIN_2015.iteritems():
    X_train_2015.append(data)
    Y_train_2015.append(gs)
X_train_2015=np.concatenate(X_train_2015,axis=0)
Y_train_2015=np.concatenate(Y_train_2015,axis=0)


print "Training shape", X_train_2015.shape
print "Training shape", Y_train_2015.shape
    


Training shape (11342, 16)
Training shape (11342,)


In [9]:
# 2015 SOPA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score


X_train_2015=np.nan_to_num(X_train_2015)
rf= RandomForestRegressor(n_estimators=100,warm_start=True)
rf.fit(X_train_2015,Y_train_2015)


for filename,(gs,data) in DATA_TEST_2015.iteritems():
    gs_=rf.predict(data)

    print "===== For",filename
    print "EVS:", explained_variance_score(gs,gs_)
    print "MAE:", mean_absolute_error(gs,gs_)
    print "MSR:", mean_squared_error(gs,gs_)
    print "mAE:", median_absolute_error(gs,gs_)
    print "R2 :", r2_score(gs,gs_)
    filename_=os.path.join("..",OUTPUT_DIR,filename)
    testfilename=os.path.join(filename)
    testfilename=testfilename.replace("STS.input","STS.2015.gs")

    fn=open(filename_,'w')
    for pred in gs_: 
        print >> fn, "{0:1.1f}".format(pred)
    fn.close()
    print "PEA:",utils.eval(CMD,os.path.join("..",TEST_2015_DIR,testfilename),filename_)


    
    #===== For STS.input.answers-forums.txt
    #PEA: 0.63331
    #===== For STS.input.images.txt
    #PEA: 0.81768
    #===== For STS.input.answers-students.txt
    #PEA: 0.63893
    #===== For STS.input.headlines.txt
    #PEA: 0.81666
    #===== For STS.input.belief.txt
    #PEA: 0.72389

 ===== For STS.input.answers-forums.txt
EVS: 0.0
MAE: 1.86811377184
MSR: 4.15697428494
mAE: 1.7635
R2 : 0.0
PEA: 0.63275
===== For STS.input.images.txt
EVS: 0.0
MAE: 3.13328328242
MSR: 11.0529142799
mAE: 3.457245
R2 : 0.0
PEA: 0.81705
===== For STS.input.answers-students.txt
EVS: 0.0
MAE: 3.10291050368
MSR: 10.4292894103
mAE: 3.271615
R2 : 0.0
PEA: 0.64689
===== For STS.input.headlines.txt
EVS: 0.0
MAE: 2.98937366237
MSR: 9.86131757636
mAE: 3.17827
R2 : 0.0
PEA: 0.81409
===== For STS.input.belief.txt
EVS: 0.0
MAE: 1.32385775252
MSR: 2.78627861894
mAE: 1.06275
R2 : 0.0
PEA: 0.72114


In [10]:
DATA_TRAIN_2016={}
DATA_TEST_2016={}
RAW_TRAIN_2016={}
RAW_TEST_2016={}

print RAW_TEST_2015.keys()

for filename,data in DATA_TRAIN_2015.iteritems():
    DATA_TRAIN_2016[filename]=data
    RAW_TRAIN_2016[filename]=RAW_TRAIN_2015[filename]
for filename,data in DATA_TEST_2015.iteritems():
    DATA_TRAIN_2016[filename]=data
    RAW_TRAIN_2016[filename]=RAW_TEST_2015[filename]
    
for filename in TEST_2016_NAMEFILES:
    filename_=make_data_name(TEST_2016_DATADIR,filename,replace=False)
    gs,data=read_data_file(filename_,)
    DATA_TEST_2016[filename]=(gs,data)
    rawname=make_raw_name(RAW_2016_TRAINDIR,filename.replace('STS.','STS.2015.'))
    cols=read_raw_file(rawname)
    RAW_TEST_2016[filename]=cols
    print "Shape para",filename,data.shape
    
print "Total train data files 2016:",len(DATA_TRAIN_2016)


    

['STS.input.answers-forums.txt', 'STS.input.images.txt', 'STS.input.answers-students.txt', 'STS.input.headlines.txt', 'STS.input.belief.txt']
Shape para STS2016.input.answer-answer.txt (1572, 16)
Shape para STS2016.input.headlines.txt (1498, 16)
Shape para STS2016.input.plagiarism.txt (1271, 16)
Shape para STS2016.input.postediting.txt (3287, 16)
Shape para STS2016.input.question-question.txt (1555, 16)
Total train data files 2016: 23


In [12]:
# 2016 SOPA
X_train_2016=[]
Y_train_2016=[]

for filename,(gs,data) in DATA_TRAIN_2016.iteritems():
    X_train_2016.append(data)
    Y_train_2016.append(gs)
X_train_2016=np.concatenate(X_train_2016,axis=0)
Y_train_2016=np.concatenate(Y_train_2016,axis=0)


print "Training shape", X_train_2016.shape
print "Training shape", Y_train_2016.shape
    

X_train_2016=np.nan_to_num(X_train_2016)
rf= RandomForestRegressor(n_estimators=100)
rf.fit(X_train_2016,Y_train_2016)


for filename,(gs,data) in DATA_TEST_2016.iteritems():
    gs_=rf.predict(data)
    print "Testing shape",filename,data.shape
    filename_=os.path.join("..",OUTPUT_DIR,filename)
    fn=open(filename_,'w')
    for pred in gs_: 
        print >> fn, "{0:1.3f}".format(pred)
    fn.close()
    

Training shape (19842, 16)
Training shape (19842,)
Testing shape STS2016.input.postediting.txt (3287, 16)
Testing shape STS2016.input.question-question.txt (1555, 16)
Testing shape STS2016.input.headlines.txt (1498, 16)
Testing shape STS2016.input.plagiarism.txt (1271, 16)
Testing shape STS2016.input.answer-answer.txt (1572, 16)


In [13]:
# 2015 + cols
X_train_2015=[]
Y_train_2015=[]
X_test_2015=[]
Y_test_2015=[]

for filename,(gs,data) in DATA_TRAIN_2015.iteritems():
    cols=RAW_TRAIN_2015[filename]
    X_train_2015.append(np.hstack((data,cols)))
    Y_train_2015.append(gs)
X_train_2015=np.concatenate(X_train_2015,axis=0)
Y_train_2015=np.concatenate(Y_train_2015,axis=0)


print "Training shape", X_train_2015.shape
print "Training shape", Y_train_2015.shape

Training shape (11342, 23)
Training shape (11342,)


In [15]:
X_train_2015=np.nan_to_num(X_train_2015)
rf= RandomForestRegressor(n_estimators=100,warm_start=True)
rf.fit(X_train_2015,Y_train_2015)


for filename,(gs,data) in DATA_TEST_2015.iteritems():
    cols=RAW_TEST_2015[filename]
    gs_=rf.predict(np.hstack((data,cols)))

    print "===== For",filename
    #print "EVS:", explained_variance_score(gs,gs_)
    #print "MAE:", mean_absolute_error(gs,gs_)
    #print "MSR:", mean_squared_error(gs,gs_)
    #print "mAE:", median_absolute_error(gs,gs_)
    #print "R2 :", r2_score(gs,gs_)
    filename_=os.path.join("..",OUTPUT_DIR,filename)
    testfilename=os.path.join(filename)
    testfilename=testfilename.replace("STS.input","STS.2015.gs")

    fn=open(filename_,'w')
    for pred in gs_: 
        print >> fn, "{0:1.1f}".format(pred)
    fn.close()
    print "PEA:",utils.eval(CMD,os.path.join("..",TEST_2015_DIR,testfilename),filename_)


#===== For STS.input.answers-forums.txt
#PEA: 0.65703
#===== For STS.input.images.txt
#PEA: 0.84109
#===== For STS.input.answers-students.txt
#PEA: 0.70356
#===== For STS.input.headlines.txt
#PEA: 0.8247
#===== For STS.input.belief.txt
#PEA: 0.73149
    

===== For STS.input.answers-forums.txt
PEA: 0.64814
===== For STS.input.images.txt
PEA: 0.83892
===== For STS.input.answers-students.txt
PEA: 0.70501
===== For STS.input.headlines.txt
PEA: 0.82177
===== For STS.input.belief.txt
PEA: 0.72456


In [16]:
# 2016 SOPA + cols
X_train_2016=[]
Y_train_2016=[]

for filename,(gs,data) in DATA_TRAIN_2016.iteritems():
    cols=RAW_TRAIN_2016[filename]
    X_train_2016.append(np.hstack((data,cols)))
    Y_train_2016.append(gs)
X_train_2016=np.concatenate(X_train_2016,axis=0)
Y_train_2016=np.concatenate(Y_train_2016,axis=0)


print "Training shape", X_train_2016.shape
print "Training shape", Y_train_2016.shape
    

X_train_2016=np.nan_to_num(X_train_2016)
rf= RandomForestRegressor(n_estimators=100)
rf.fit(X_train_2016,Y_train_2016)


for filename,(gs,data) in DATA_TEST_2016.iteritems():

    cols=RAW_TEST_2016[filename]
    data=np.hstack((data,cols))
    data=np.nan_to_num(data)
    gs_=rf.predict(data)
    print "Testing shape",filename,data.shape
    filename_=os.path.join("..",OUTPUT_DIR,filename)
    fn=open(filename_,'w')
    for pred in gs_: 
        print >> fn, "{0:1.3f}".format(pred)
    fn.close()

Training shape (19842, 23)
Training shape (19842,)
Testing shape STS2016.input.postediting.txt (3287, 23)
Testing shape STS2016.input.question-question.txt (1555, 23)
Testing shape STS2016.input.headlines.txt (1498, 23)
Testing shape STS2016.input.plagiarism.txt (1271, 23)
Testing shape STS2016.input.answer-answer.txt (1572, 23)


In [17]:
CROSS_TEST_2016={}


for filename in CROSS_2016_NAMEFILES:
    filename_=make_data_name(CROSS_DIR,filename,replace=False)
    gs,data=read_data_file(filename_)
    CROSS_TEST_2016[filename]=(gs,data)
    print "Shape para",filename,data.shape

Shape para STS.2016.input.crosslingual-news-part1.txt (301, 16)
Shape para STS.2016.input.crosslingual-news-part2.txt (2973, 16)


In [19]:
# 2016 SOPA -> CROSS
X_train_2016=[]
Y_train_2016=[]

for filename,(gs,data) in DATA_TRAIN_2016.iteritems():
    X_train_2016.append(data)
    Y_train_2016.append(gs)
X_train_2016=np.concatenate(X_train_2016,axis=0)
Y_train_2016=np.concatenate(Y_train_2016,axis=0)


print "Training shape", X_train_2016.shape
print "Training shape", Y_train_2016.shape
    

X_train_2016=np.nan_to_num(X_train_2016)
rf= RandomForestRegressor(n_estimators=100)
rf.fit(X_train_2016,Y_train_2016)


for filename,(gs,data) in CROSS_TEST_2016.iteritems():
    gs_=rf.predict(data)
    print "Testing shape",filename,data.shape
    filename_=os.path.join("..",OUTPUT_DIR,filename)
    fn=open(filename_,'w')
    for pred in gs_: 
        print >> fn, "{0:1.3f}".format(pred)
    fn.close()


Training shape (19842, 16)
Training shape (19842,)
Testing shape STS.2016.input.crosslingual-news-part2.txt (2973, 16)
Testing shape STS.2016.input.crosslingual-news-part1.txt (301, 16)


In [71]:
# Eval 2016

SYSTEMS=["SOPA","SOPA100","SOPA1000"]
SYSTEMS_DIR="english_testbed/analysis/2016/output/"


results={}
for system in SYSTEMS:
    for filename,(gs,data) in DATA_TEST_2016.iteritems():
        filename_=filename.replace('.input.','.gs.')
        filenamegs=os.path.join("..",TEST_2016_DIR,filename_)
        filenamesys=os.path.join("..",SYSTEMS_DIR,system,filename_.replace(".gs.",".OUTPUT.LIPN-IIMAS."+system+"."))
        res=utils.eval(CMD,filenamegs,filenamesys)
        try:
            results[filename]
        except KeyError:
            results[filename]={}
        
        gs,sys=read_raw_files(filenamegs,filenamesys)
        try:
            results[filename]['PEA'].append(res)
        except KeyError:
            results[filename]['PEA']=[res]
        try:
            results[filename]['EVS'].append(explained_variance_score(gs,sys))
        except KeyError:
            results[filename]['EVS']=[explained_variance_score(gs,sys)]
        try:
            results[filename]['MAE'].append(mean_absolute_error(gs,sys))
        except KeyError:
            results[filename]['MAE']=[mean_absolute_error(gs,sys)]
        try:
            results[filename]['MSR'].append(mean_absolute_error(gs,sys))
        except KeyError:
            results[filename]['MSR']=[mean_absolute_error(gs,sys)]
        try:
            results[filename]['mAE'].append(median_absolute_error(gs,sys))
        except KeyError:
            results[filename]['mAE']=[median_absolute_error(gs,sys)]
        try:
            results[filename]['r2 '].append(r2_score(gs,sys))
        except KeyError:
            results[filename]['r2 ']=[r2_score(gs,sys)]
                              
for filename in results.keys():
        print ">>>",filename
        for name,vals in results[filename].iteritems():
            print name, ",  ".join(["{0:2.4f}".format(val) for val in vals])

>>> STS2016.input.postediting.txt
MSR 0.7924,  0.8918,  0.8787
r2  0.6370,  0.5561,  0.5640
mAE 0.6150,  0.7420,  0.7450
MAE 0.7924,  0.8918,  0.8787
EVS 0.6371,  0.5709,  0.5799
PEA 0.7986,  0.7556,  0.7616
>>> STS2016.input.answer-answer.txt
MSR 1.3409,  1.3771,  1.3726
r2  0.1714,  0.1235,  0.1418
mAE 1.2930,  1.2375,  1.2125
MAE 1.3409,  1.3771,  1.3726
EVS 0.2015,  0.1861,  0.2015
PEA 0.4490,  0.4322,  0.4489
>>> STS2016.input.headlines.txt
MSR 1.2374,  1.2440,  1.2439
r2  0.1449,  0.1148,  0.1217
mAE 0.9920,  1.0000,  0.9760
MAE 1.2374,  1.2440,  1.2439
EVS 0.3633,  0.3293,  0.3400
PEA 0.6241,  0.5850,  0.5972
>>> STS2016.input.plagiarism.txt
MSR 1.0200,  1.0621,  1.0474
r2  0.3987,  0.3538,  0.3787
mAE 0.8270,  0.8765,  0.8300
MAE 1.0200,  1.0621,  1.0474
EVS 0.4775,  0.5488,  0.5639
PEA 0.6911,  0.7473,  0.7594
>>> STS2016.input.question-question.txt
MSR 1.0238,  1.0575,  1.0504
r2  0.3319,  0.3043,  0.3136
mAE 0.9230,  1.0580,  1.0430
MAE 1.0238,  1.0575,  1.0504
EVS 0.3484,  